![XKCD data pipeline](https://imgs.xkcd.com/comics/data_pipeline.png)

[https://xkcd.com/2054](https://xkcd.com/2054/)

1. http://www.nhc.noaa.gov/gis/
2. https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/

https://www.nhc.noaa.gov/gis/archive_besttrack.php?year=2018

NHC codes storms are coded with 8 letter names:
- 2 char for region `al` &rarr; Atlantic
- 2 char for number `11` is Irma
- and 4 char for year, `2017`

Browse http://www.nhc.noaa.gov/gis/archive_wsurge.php?year=2017 to find other hurricanes code.

In [1]:
import os

import geopandas

import wget


def load_best_track(code='al14', year='2018'):
    fname = f'{code}{year}_best_track.zip'
    url = f'https://www.nhc.noaa.gov/gis/best_track/{fname}'

    if not os.path.isfile(fname):
        import wget
        fname = wget.download(url)

    os.environ['CPL_ZIP_ENCODING'] = 'UTF-8'

    radii = geopandas.read_file(
        f'/{code.upper()}{year}_radii.shp',
        vfs='zip://{}'.format(fname)
    )

    pts = geopandas.read_file(
        f'/{code.upper()}{year}_pts.shp',
        vfs='zip://{}'.format(fname)
    )
    return radii, pts


radii, pts = load_best_track(code='al14', year='2018')
bbox = tuple(radii['geometry'].total_bounds)

In [2]:
pts.columns

Index(['STORMNAME', 'DTG', 'YEAR', 'MONTH', 'DAY', 'HHMM', 'MSLP', 'BASIN',
       'STORMNUM', 'STORMTYPE', 'INTENSITY', 'SS', 'LAT', 'LON', 'geometry'],
      dtype='object')

In [3]:
import shapely


coords = zip(pts['LON'], pts['LAT'])
track = shapely.geometry.LineString(coords)

Now we can get all the information we need from those GIS files. Let's start with the event dates.

In [4]:
import pandas as pd


pts['str'] = pts['DTG'].astype(int).astype(str)

pts.index = pd.to_datetime(
    pts['str'], format='%Y%m%d%H', errors='coerce').values

In [5]:
start = pts.index[0]
end = pts.index[-1]

And the bounding box to search the data.

In [6]:
strbbox = ', '.join(format(v, '.2f') for v in bbox)
print('bbox: {}\nstart: {}\n  end: {}'.format(strbbox, start, end))

bbox: -89.23, 15.71, -69.99, 38.80
start: 2018-10-07 06:00:00
  end: 2018-10-12 06:00:00


Now we need to build a filter with those parameters to find the observations along the Hurricane path. We still need to specify:

- the units for the observations;
- and the SOS name for the variables of interest.

Next, we can use `pyoos` to assemble a collector to download the data into a pandas `DataFrame`.

In [7]:
import cf_units
from ioos_tools.ioos import collector2table
import pandas as pd
from pyoos.collectors.coops.coops_sos import CoopsSos
from retrying import retry


# We need to retry in case of failure b/c the server cannot handle
# the high traffic during events like Irma.
@retry(stop_max_attempt_number=5, wait_fixed=3000)
def get_coops(start, end, sos_name, units, bbox, verbose=False):
    collector = CoopsSos()
    collector.set_bbox(bbox)
    collector.end_time = end
    collector.start_time = start
    collector.variables = [sos_name]
    ofrs = collector.server.offerings
    title = collector.server.identification.title
    config = dict(
        units=units,
        sos_name=sos_name,
    )

    data = collector2table(
        collector=collector,
        config=config,
        col='{} ({})'.format(sos_name, units.format(cf_units.UT_ISO_8859_1))
    )

    # Clean the table.
    table = dict(
        station_name=[s._metadata.get('station_name') for s in data],
        station_code=[s._metadata.get('station_code') for s in data],
        sensor=[s._metadata.get('sensor') for s in data],
        lon=[s._metadata.get('lon') for s in data],
        lat=[s._metadata.get('lat') for s in data],
        depth=[s._metadata.get('depth', 'NA') for s in data],
    )

    table = pd.DataFrame(table).set_index('station_name')
    if verbose:
        print('Collector offerings')
        print('{}: {} offerings'.format(title, len(ofrs)))
    return data, table

In [8]:
ssh, ssh_table = get_coops(
    start=start,
    end=end,
    sos_name='water_surface_height_above_reference_datum',
    units=cf_units.Unit('meters'),
    bbox=bbox,
)

ssh_table

,station_code,sensor,lon,lat,depth
station_name,,,,,
"Lewes, DE",8557380,urn:ioos:sensor:NOAA.NOS.CO-OPS:8557380:A1,-75.1192,38.7828,None
"Ocean City Inlet, MD",8570283,urn:ioos:sensor:NOAA.NOS.CO-OPS:8570283:A1,-75.0911,38.3283,None
"Bishops Head, MD",8571421,urn:ioos:sensor:NOAA.NOS.CO-OPS:8571421:A1,-76.0387,38.2204,None
"Cambridge, MD",8571892,urn:ioos:sensor:NOAA.NOS.CO-OPS:8571892:Y1,-76.0722,38.5742,None
"Solomons Island, MD",8577330,urn:ioos:sensor:NOAA.NOS.CO-OPS:8577330:Y1,-76.4508,38.3172,None
"Wachapreague, VA",8631044,urn:ioos:sensor:NOAA.NOS.CO-OPS:8631044:Y1,-75.6858,37.6078,None
"Kiptopeke, VA",8632200,urn:ioos:sensor:NOAA.NOS.CO-OPS:8632200:A1,-75.9884,37.1652,None
"Dahlgren, VA",8635027,urn:ioos:sensor:NOAA.NOS.CO-OPS:8635027:Y1,-77.0366,38.3197,None
"Lewisetta, VA",8635750,urn:ioos:sensor:NOAA.NOS.CO-OPS:8635750:A1,-76.4646,37.9954,None


In [9]:
wind_speed, wind_speed_table = get_coops(
    start=start,
    end=end,
    sos_name='wind_speed',
    units=cf_units.Unit('m/s'),
    bbox=bbox,
)

wind_speed_table

,station_code,sensor,lon,lat,depth
station_name,,,,,
"Lewes, DE",8557380,urn:ioos:sensor:NOAA.NOS.CO-OPS:8557380:C1,-75.1192,38.7828,None
"Ocean City Inlet, MD",8570283,urn:ioos:sensor:NOAA.NOS.CO-OPS:8570283:C1,-75.0917,38.3283,None
"Bishops Head, MD",8571421,urn:ioos:sensor:NOAA.NOS.CO-OPS:8571421:C1,-76.0387,38.2204,None
"Cove Point LNG Pier, MD",8577018,urn:ioos:sensor:NOAA.NOS.CO-OPS:8577018:C1,-76.3855,38.4044,None
"Solomons Island, MD",8577330,urn:ioos:sensor:NOAA.NOS.CO-OPS:8577330:C1,-76.4508,38.3172,None
"Piney Point, MD",8578240,urn:ioos:sensor:NOAA.NOS.CO-OPS:8578240:C1,-76.5333,38.1333,None
"Wachapreague, VA",8631044,urn:ioos:sensor:NOAA.NOS.CO-OPS:8631044:C1,-75.6858,37.6078,None
"Kiptopeke, VA",8632200,urn:ioos:sensor:NOAA.NOS.CO-OPS:8632200:C1,-75.9884,37.1652,None
"Rappahannock Light, VA",8632837,urn:ioos:sensor:NOAA.NOS.CO-OPS:8632837:C1,-76.0150,37.5383,None


For simplicity we will use only the stations that have both wind speed and sea surface height and reject those that have only one or the other.

In [10]:
common = set(ssh_table['station_code']).intersection(wind_speed_table['station_code'])

In [11]:
ssh_obs, win_obs = [], []
for station in common:
    ssh_obs.extend([obs for obs in ssh if obs._metadata['station_code'] == station])
    win_obs.extend([obs for obs in wind_speed if obs._metadata['station_code'] == station])

In [12]:
index = pd.date_range(
    start=start.replace(tzinfo=None),
    end=end.replace(tzinfo=None),
    freq='15min'
)

# Re-index and rename series.
ssh_observations = []
for series in ssh_obs:
    _metadata = series._metadata
    obs = series.reindex(index=index, limit=1, method='nearest')
    obs._metadata = _metadata
    obs.name = _metadata['station_name']
    ssh_observations.append(obs)

winds_observations = []
for series in win_obs:
    _metadata = series._metadata
    obs = series.reindex(index=index, limit=1, method='nearest')
    obs._metadata = _metadata
    obs.name = _metadata['station_name']
    winds_observations.append(obs)

Let's take a look at some stations to see if the data is OK. Below we have a station in Naples, FL along the Gulf of Mexico.

We can observe the sea level retreating around 10-Sep 9:00 and then a significant surge after 19:00.
The lower winds at beginning of the surge is probably the eye of the hurricane.

For our interactive map we will use [`bokeh`](https://bokeh.pydata.org/en/latest) HTML plots instead of the usual raster [`matplotlib`](https://matplotlib.org) ones to enhance the user experience when exploring the graphs.

In [13]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.models import Range1d, LinearAxis, HoverTool

from folium import IFrame

# Plot defaults.
tools = "pan,box_zoom,reset"
width, height = 750, 250


def make_plot(ssh, wind):
    p = figure(toolbar_location='above',
               x_axis_type='datetime',
               width=width,
               height=height,
               tools=tools,
               title=ssh.name)

    p.yaxis.axis_label = 'wind speed (m/s)'

    l0 = p.line(
        x=wind.index,
        y=wind.values,
        line_width=5,
        line_cap='round',
        line_join='round',
        legend='wind speed (m/s)',
        color='#9900cc',
        alpha=0.5,
    )

    p.extra_y_ranges = {}
    p.extra_y_ranges['y2'] = Range1d(
        start=-1,
        end=3.5
    )

    p.add_layout(
        LinearAxis(
            y_range_name='y2',
            axis_label='ssh (m)'),
        'right'
    )

    l1 = p.line(
        x=ssh.index,
        y=ssh.values,
        line_width=5,
        line_cap='round',
        line_join='round',
        legend='ssh (m)',
        color='#0000ff',
        alpha=0.5,
        y_range_name='y2',
    )

    p.legend.location = 'top_left'

    p.add_tools(
        HoverTool(
            tooltips=[
                ('wind speed (m/s)', '@y'),
            ],
            renderers=[l0],
        ),
        HoverTool(
            tooltips=[
                ('ssh (m)', '@y'),
            ],
            renderers=[l1],
        ),
    )
    return p


def make_marker(p, location, fname):
    html = file_html(p, CDN, fname)
    iframe = IFrame(html, width=width+45, height=height+80)

    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='green', icon='stats')
    marker = folium.Marker(location=location,
                           popup=popup,
                           icon=icon)
    return marker

Here is the final result. Explore the map by clicking on the map features plotted!

In [14]:
import folium
from folium.plugins import Fullscreen, MarkerCluster
from ioos_tools.ioos import get_coordinates


lon = track.centroid.x
lat = track.centroid.y

m = folium.Map(location=[lat, lon], tiles='OpenStreetMap', zoom_start=4)

Fullscreen(position='topright', force_separate_button=True).add_to(m)

marker_cluster0 = MarkerCluster(name='Observations')
marker_cluster0.add_to(m);

In [15]:
url = 'http://oos.soest.hawaii.edu/thredds/wms/hioos/satellite/dhw_5km'
w0 = folium.WmsTileLayer(
    url,
    name='Sea Surface Temperature',
    fmt='image/png',
    layers='CRW_SST',
    attr='PacIOOS TDS',
    overlay=True,
    transparent=True)

url = 'http://hfrnet.ucsd.edu/thredds/wms/HFRNet/USEGC/6km/hourly/RTV'
w1 = folium.WmsTileLayer(
    url,
    name='HF Radar',
    fmt='image/png',
    layers='surface_sea_water_velocity',
    attr='HFRNet',
    overlay=True,
    transparent=True)

w0.add_to(m)
w1.add_to(m);

In [16]:
colors = {
    'EX': 'yellow',
    'TD': 'yellow',
    'TS': 'orange',
    'HU': 'red',
}

In [17]:
def style_function(feature):
    return {
        'fillOpacity': 0,
        'color': 'black',
        'stroke': 1,
        'weight': 0.5,
        'opacity': 0.2,
    }


for date, row in pts.iterrows():
    storm_type = row['STORMTYPE']
    location = row['LAT'], row['LON']
    popup = '{}<br>{}'.format(date, storm_type)
    folium.CircleMarker(
        location=location,
        radius=10,
        fill=True,
        color=colors[storm_type],
        popup=popup,
    ).add_to(m)

In [18]:
# Observations.
for ssh, wind in zip(ssh_observations, winds_observations):
    fname = ssh._metadata['station_code']
    location = ssh._metadata['lat'], ssh._metadata['lon']
    p = make_plot(ssh, wind)
    marker = make_marker(p, location=location, fname=fname)
    marker.add_to(marker_cluster0)

folium.LayerControl().add_to(m)

p = folium.PolyLine(get_coordinates(bbox),
                    color='#009933',
                    weight=1,
                    opacity=0.2)

p.add_to(m);

In [19]:
def embed_map(m):
    from IPython.display import HTML

    m.save('index.html')
    with open('index.html') as f:
        html = f.read()

    iframe = '<iframe srcdoc="{srcdoc}" style="width: 100%; height: 750px; border: none"></iframe>'
    srcdoc = html.replace('"', '&quot;')
    return HTML(iframe.format(srcdoc=srcdoc))


embed_map(m)